# **Markov models **
### *HIDDEN MARKOV MODEL (HMM) IS SET OF ALGORITHMS FOR UNSUPERVISED LEARNING AND INFERENCE OF HIDDEN MARKOV MODELS*

The components of an HMM include:


*   Hidden States: The actual states of the model, which are not directly observable.
*   Observations: The visible outputs, which are influenced by the hidden states.
*   Transition Probabilities: As in Markov Chains, these describe the chance of transitioning from one hidden state to another.
*   Emission Probabilities: These probabilities model the likelihood of observing a particular visible state given a hidden state.
*   Initial State Probabilities: These are the probabilities of starting in a particular state.

**Installing required libraries by pip**

In [19]:
!pip install hmmlearn
!pip install nltk editdistance

## USING HIDDEN MARKOV MODEL TO MAKE SPELLING CORRECTOR

In [20]:
import nltk
import numpy as np
import editdistance
from collections import defaultdict, Counter
from nltk.corpus import brown
nltk.download('brown')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

## Data Preparation Steps:

*   Load Brown corpus as training data.
*   Preprocess (lowercase, remove punctuation if needed).
*   Create vocabulary.



In [21]:
sentences = brown.sents(categories='news')
sentences = [[w.lower() for w in sent] for sent in sentences]

vocab = sorted(set(w for sent in sentences for w in sent))
vocab_index = {w: i for i, w in enumerate(vocab)}


In [26]:
vocab_index

{'!': 0,
 '$1': 1,
 '$1,000': 2,
 '$1,000,000,000': 3,
 '$1,500': 4,
 '$1,500,000': 5,
 '$1,600': 6,
 '$1,800': 7,
 '$1.1': 8,
 '$1.4': 9,
 '$1.5': 10,
 '$1.80': 11,
 '$10': 12,
 '$10,000': 13,
 '$10,000-per-year': 14,
 '$100': 15,
 '$100,000': 16,
 '$102,285,000': 17,
 '$109': 18,
 '$11.50': 19,
 '$115,000': 20,
 '$12': 21,
 '$12,192,865': 22,
 '$12,500': 23,
 '$12.50': 24,
 '$12.7': 25,
 '$120': 26,
 '$125': 27,
 '$135': 28,
 '$139.3': 29,
 '$14': 30,
 '$15': 31,
 '$15,000': 32,
 '$15,000,000': 33,
 '$150': 34,
 '$157,460': 35,
 '$16': 36,
 '$16,000': 37,
 '$17': 38,
 '$17,000': 39,
 '$17.8': 40,
 '$172,000': 41,
 '$172,400': 42,
 '$18': 43,
 '$18.2': 44,
 '$18.9': 45,
 '$2': 46,
 '$2,000': 47,
 '$2,170': 48,
 '$2,330,000': 49,
 '$2,700': 50,
 '$2.50': 51,
 '$2.80': 52,
 '$20': 53,
 '$20,000': 54,
 '$20,447,000': 55,
 '$200,000': 56,
 '$214': 57,
 '$22': 58,
 '$22.50': 59,
 '$2400': 60,
 '$25': 61,
 '$25,000': 62,
 '$25-a-plate': 63,
 '$250': 64,
 '$250,000': 65,
 '$251': 66,
 '$253,

# Build HMM Components

In [27]:
# transition models

bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

for sent in sentences:
    prev = "<s>"
    unigram_counts[prev] += 1
    for word in sent:
        unigram_counts[word] += 1
        bigram_counts[(prev, word)] += 1
        prev = word
    bigram_counts[(prev, "</s>")] += 1

def transition_prob(w1, w2):
    return (bigram_counts[(w1, w2)] + 1) / (unigram_counts[w1] + len(vocab))


In [28]:
# emission probabilities

def emission_prob(obs, true_word):
    dist = editdistance.eval(obs, true_word)
    if dist == 0: return 0.9
    elif dist == 1: return 0.05
    elif dist == 2: return 0.03
    else: return 1e-6


##  Candidate Generation (Speed Optimization)
*  Instead of checking all words in vocab, we only consider those with small edit distance.





In [29]:
def generate_candidates(obs, max_dist=2):
    return [w for w in vocab if editdistance.eval(obs, w) <= max_dist]


## Viterbi algorithm for spelling correction

In [30]:
def viterbi_spell(obs_sentence):
    candidates_list = [generate_candidates(obs) for obs in obs_sentence]
    V = [{}]
    path = {}

    # Init
    for cand in candidates_list[0]:
        V[0][cand] = np.log(transition_prob("<s>", cand)) + np.log(emission_prob(obs_sentence[0], cand))
        path[cand] = [cand]

    # DP
    for t in range(1, len(obs_sentence)):
        V.append({})
        new_path = {}
        for cand in candidates_list[t]:
            (prob, prev_cand) = max(
                (V[t-1][pc] + np.log(transition_prob(pc, cand)) + np.log(emission_prob(obs_sentence[t], cand)), pc)
                for pc in candidates_list[t-1]
            )
            V[t][cand] = prob
            new_path[cand] = path[prev_cand] + [cand]
        path = new_path

    # End
    (prob, last_cand) = max((V[-1][c] + np.log(transition_prob(c, "</s>")), c) for c in candidates_list[-1])
    return path[last_cand]


In [33]:
misspelled = ["thhe", "doog", "breks"]
print("Observed:", misspelled)
print("Corrected:", viterbi_spell(misspelled))


Observed: ['thhe', 'doog', 'breks']
Corrected: ['the', 'door', 'breaks']
